In [2]:
import os
os.chdir("/Users/ericsmac/Documents/MATH300Project/data")

In [3]:


import pandas as pd 

df = pd.read_feather('ml_market_data.ft')


In [4]:

df['target_return_20d']=df.groupby('symbol')['target_return_1d'].shift(19)

df = df.dropna()
print(df.columns)


Index(['time', 'symbol', 'open', 'close', 'high', 'low', 'trade_count',
       'volume', 'vwap', 'return', 'vol_t', 'vol_lag1', 'vol_lag2', 'vol_lag3',
       'vol_lag4', 'vol_lag5', 'vol_lag20', 'ret_lag1', 'ret_lag2', 'ret_lag3',
       'ret_lag4', 'ret_lag5', 'ret_lag10', 'vol_roll_mean_5',
       'vol_roll_std_20', 'vol_roll_mean_20', 'vol_roll_std_5', 'dow', 'month',
       'target_return_1d', 'target_return_20d'],
      dtype='object')


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
import numpy as np
from scipy.stats import spearmanr
from sklearn.metrics import mean_absolute_error

# -----------------------
# 1. Select features
# -----------------------
feature_cols = ['time', 'open', 'close', 'high', 'low', 'trade_count',
       'volume', 'vwap', 'return', 'vol_t', 'vol_lag1', 'vol_lag2', 'vol_lag3',
       'vol_lag4', 'vol_lag5', 'vol_lag20', 'ret_lag1', 'ret_lag2', 'ret_lag3',
       'ret_lag4', 'ret_lag5', 'ret_lag10', 'vol_roll_mean_5',
       'vol_roll_std_20', 'vol_roll_mean_20', 'vol_roll_std_5', 'dow', 'month',
       'target_return_20d']
df_sort = df[feature_cols]

# -----------------------
# 2. Time-series split
# -----------------------
df_sorted = df_sort.sort_values("time").reset_index(drop=True)

split_idx = int(len(df_sorted) * 0.8)

train = df_sorted.iloc[:split_idx]
test  = df_sorted.iloc[split_idx:]

X_train = train.drop(columns=["target_return_20d", "time"])
y_train = train["target_return_20d"]

X_test  = test.drop(columns=["target_return_20d", "time"])
y_test  = test["target_return_20d"]


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# -----------------------
# 4. XGBoost model
# -----------------------
xgb_model = XGBRegressor(
    n_estimators=2000,           # number of trees
    learning_rate=0.01,         # shrinkage
    max_depth=8,                # tree depth (controls complexity)
    subsample=0.8,              # row subsample
    colsample_bytree=0.8,       # feature subsample per tree
    objective="reg:squarederror",
    random_state=42
)

xgb_model.fit(X_train_scaled, y_train)

# -----------------------
# 5. Predictions
# -----------------------
y_pred_train = xgb_model.predict(X_train_scaled)
y_pred_test  = xgb_model.predict(X_test_scaled)

# MAE
mae = mean_absolute_error(y_test, y_pred_test)

# RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

# MAPE (avoid division by zero)
mape = np.mean(np.abs((y_test - y_pred_test) / (y_test + 1e-9))) * 100

# Directional accuracy
direction_true = np.sign(y_test)
direction_pred = np.sign(y_pred_test)
direction_accuracy = np.mean(direction_true == direction_pred)

print("----- Accuracy Metrics -----")
print("MAE :", mae)
print("RMSE:", rmse)
print("MAPE:", mape, "%")
print("Directional Accuracy:", direction_accuracy)

----- Accuracy Metrics -----
MAE : 0.032150356234799594
RMSE: 0.711732209642146
MAPE: 55163059.58144226 %
Directional Accuracy: 0.475347397821467
